# This Code does the DyCA Analysis of the sEEG Data from EBRAINS (Amir)

- First try to handle the data in the propper manner (xarray)


In [21]:
import numpy as np
import pandas as pd
import xarray as xr
import dyca

In [22]:
# test_array = xr.load_dataarray("/home/astiehl/05_Code/Promotion/marseille/IREM/seeg_epochs/HID-Sub-001_MVIS_MVEB_Epochs.nc")
test_array = xr.load_dataarray("/home/astiehl/05_Code/Promotion/marseille/IREM/seeg_epochs/new_data/test1.nc")
# count nans and notnans in the data
nans = np.isnan(test_array).sum().item()
notnans = np.count_nonzero(~np.isnan(test_array))

print(f"Number of NaNs in the data array: {nans}")
print(f"Number of non-NaNs in the data array: {notnans}")

# maybe i can reconstruct the number of datapoints
total_points = nans + notnans
total_points2 = test_array.size
total_points3 = 256*93*2*2*2*36
print(f"Total number of data points (nans):n): {total_points}")
print(f"Total number of data points (size): {total_points2}")
print(f"Total number of data points (manual calc): {total_points3}")
# JUHU - thats the same and I understood the data structure

# Where happens the NaNs?
# Get unique combinations of task, band, event, trial and region where NaNs are located (excluding time)
nan_mask = np.isnan(test_array)
# Check if any time point has NaN for each combination of other dimensions
nan_combinations = nan_mask.any(dim='times')
nan_indices = np.argwhere(nan_combinations.values)

unique_combinations = set()
for loc in nan_indices:
    task, band, event, trial, region = loc
    combination = (task, band, event, trial, region)
    unique_combinations.add(combination)

i = 0
for task, band, event, trial, region in sorted(unique_combinations):
    print(f"{i}. NaN found at - Task: {test_array.task[task].item()}, Band: {test_array.band[band].item()}, Event: {test_array.event[event].item()}, Trial: {test_array.trial[trial].item()}, Region: {test_array.region[region].item()}")
    i += 1
# Okay, when a NaN is present, it is present for all timepoints in that combination of other dimensions

# Print unique values for each dimension where NaNs are located
unique_tasks = set()
unique_bands = set()
unique_events = set()
unique_trials = set()
unique_regions = set()

for task, band, event, trial, region in unique_combinations:
    unique_tasks.add(test_array.task[task].item())
    unique_bands.add(test_array.band[band].item())
    unique_events.add(test_array.event[event].item())
    unique_trials.add(test_array.trial[trial].item())
    unique_regions.add(test_array.region[region].item())

print("\nUnique values where NaNs are found:")
print(f"Tasks: {sorted(unique_tasks)}")
print(f"Bands: {sorted(unique_bands)}")
print(f"Events: {sorted(unique_events)}")
print(f"Trials: {sorted(unique_trials)}")
print(f"Regions: {sorted(unique_regions)}")
# here you can see, that all the Nans happens in the Trial 35 
    
test_array

Number of NaNs in the data array: 0
Number of non-NaNs in the data array: 4158720
Total number of data points (nans):n): 4158720
Total number of data points (size): 4158720
Total number of data points (manual calc): 6856704

Unique values where NaNs are found:
Tasks: []
Bands: []
Events: []
Trials: []
Regions: []


<xarray.DataArray 'f50f150' (trials: 95, roi: 114, times: 384)> Size: 17MB
array([[[7.24609345e-05, 8.60351574e-05, 9.81445337e-05, ...,
         8.02734357e-05, 7.95898450e-05, 8.01757778e-05],
        [8.94531258e-05, 9.75585936e-05, 1.05761719e-04, ...,
         1.11230467e-04, 1.07812499e-04, 1.01562502e-04],
        [7.13867194e-05, 6.93359398e-05, 7.04101549e-05, ...,
         7.89062469e-05, 6.47460911e-05, 5.98632796e-05],
        ...,
        [7.98828114e-05, 9.26757784e-05, 1.02539059e-04, ...,
         7.96875029e-05, 8.96484344e-05, 9.67773449e-05],
        [8.00781272e-05, 8.65234397e-05, 9.95117152e-05, ...,
         7.61718766e-05, 8.37890620e-05, 8.79882791e-05],
        [1.02929684e-04, 1.01953126e-04, 9.94140646e-05, ...,
         7.76367160e-05, 7.49999963e-05, 7.32421904e-05]],

       [[9.97070310e-05, 9.16015633e-05, 8.75976548e-05, ...,
         7.50976542e-05, 7.10937529e-05, 6.41601582e-05],
        [1.04687497e-04, 9.90234403e-05, 1.00976562e-04, ...,
         8.77929706e-05, 9.27734363e-05, 9.66796870e-05],
        [8.53515594e-05, 8.11523423e-05, 7.37304654e-05, ...,
         1.15722658e-04, 1.10156252e-04, 1.03320315e-04],
...
        [1.10058594e-04, 1.07617190e-04, 9.97070310e-05, ...,
         1.05859377e-04, 1.20214841e-04, 1.33105466e-04],
        [1.19824217e-04, 1.06933592e-04, 8.93554679e-05, ...,
         8.48632844e-05, 9.00390587e-05, 9.48242159e-05],
        [7.92968713e-05, 7.75390654e-05, 7.93945292e-05, ...,
         6.85546838e-05, 7.28515588e-05, 7.29492167e-05]],

       [[1.05859377e-04, 9.99023468e-05, 9.39453093e-05, ...,
         1.26953120e-04, 1.35546870e-04, 1.37207026e-04],
        [1.00878904e-04, 9.67773449e-05, 8.97460923e-05, ...,
         8.81835949e-05, 8.36914041e-05, 8.06640601e-05],
        [8.61328153e-05, 9.28710942e-05, 1.05566403e-04, ...,
         9.49218738e-05, 1.01171878e-04, 1.04589846e-04],
        ...,
        [9.51171896e-05, 9.57031225e-05, 9.88281245e-05, ...,
         9.54101561e-05, 9.80468758e-05, 1.04101564e-04],
        [1.06738284e-04, 1.07128908e-04, 1.11816407e-04, ...,
         9.51171896e-05, 1.04199222e-04, 1.12109374e-04],
        [1.49023443e-04, 1.42480465e-04, 1.35742186e-04, ...,
         1.23730468e-04, 1.26757819e-04, 1.33691399e-04]]], dtype=float32)
Coordinates:
  * trials         (trials) object 760B 'hard' 'hard' 'hard' ... 'hard' 'easy'
  * roi            (roi) object 912B 'R_Insula' ... 'not in a mars atlas parcel'
  * times          (times) float64 3kB -2.0 -1.984 -1.969 ... 3.953 3.969 3.984
    contacts       (roi) object 912B 't02' 't03' 't04' ... 'f08' 'f09' 'f10'
    T_orientation  (trials) object 760B 'T up' 'T down' 'T up' ... 'T up' 'T up'
    difficulty     (trials) object 760B 'hard' 'hard' 'hard' ... 'hard' 'easy'
    response_time  (trials) float64 760B 1.094 1.578 1.781 ... 0.7031 0.5938
    correct        (trials) bool 95B True True True True ... True True True True
Attributes:
    sfreq:             64.0
    reference:         sample_stim
    unit:              V
    smoothing:         sm0
    n_samples_before:  128
    n_samples_after:   256
    freq:              f50f150
    subject:           HID-Sub-004

## Ready with playing with the data - Let's start some analysis

- group the data accordingly:
    - task, band, event, feedback
- let's run some dyca

In [23]:
# delete trial 35
cleaned_array = test_array.sel(trials=test_array.trials != 35)

# count nans and notnans in the cleaned data
nans_cleaned = np.isnan(cleaned_array).sum().item()
notnans_cleaned = np.count_nonzero(~np.isnan(cleaned_array))
print(f"Number of NaNs in the cleaned data array: {nans_cleaned}")
print(f"Number of non-NaNs in the cleaned data array: {notnans_cleaned}")

Number of NaNs in the cleaned data array: 0
Number of non-NaNs in the cleaned data array: 4158720


In [24]:
# clean the data respectively deleting regions: not in a mars atlas parcel
# cleaned_array = cleaned_array.sel(region=cleaned_array.region != "not in a mars atlas parcel")
print(cleaned_array.roi)


<xarray.DataArray 'roi' (roi: 114)> Size: 912B
array(['R_Insula', 'R_Insula', 'R_STCc', 'R_STCc', 'R_STCc', 'R_STCc',
       'R_STCc', 'R_VCl', 'R_VCl', 'R_VCl', 'R_VCs', 'R_VCs', 'R_VCl', 'R_VCl',
       'R_MCC', 'R_MCC', 'R_MCC', 'R_ICC', 'not in a mars atlas parcel',
       'not in a mars atlas parcel', 'not in a mars atlas parcel', 'R_Sdl',
       'R_Sdl', 'R_Sdl', 'R_Sdl', 'R_Sdl', 'R_Sdl', 'R_IPCv',
       'not in a mars atlas parcel', 'R_MCC', 'R_MCC', 'R_MCC',
       'not in a mars atlas parcel', 'R_PMdl', 'R_PMdl', 'R_PMdl', 'R_PMdl',
       'R_Mv', 'R_Mv', 'R_Mv', 'R_Mv', 'R_Mv', 'R_ACC', 'R_OFCvl', 'R_OFCvl',
       'R_OFCvl', 'R_OFCvl', 'R_PFrvl', 'R_PFrvl', 'R_PFrvl', 'R_PFrvl',
       'R_PFrvl', 'R_PFrvl', 'R_PFrvl', 'R_PFrvl', 'R_PFrvl', 'R_ICC', 'R_ICC',
       'R_ITCr', 'R_VCcm', 'R_VCl', 'R_VCl', 'R_MTCc', 'R_MTCc', 'R_MTCc',
       'R_MTCc', 'R_MTCc', 'R_IPCv', 'not in a mars atlas parcel', 'R_IPCv',
       'R_IPCv', 'R_IPCv', 'R_IPCv', 'R_IPCv', 'R_IPCv', 'R_IPCv', 

In [29]:
data = cleaned_array.to_numpy()

print(data.shape)
for trial in range(data.shape[0]):
    data_cut = data[trial,:,:128]
    print("Rank of the data array before DyCA:", np.linalg.matrix_rank(data_cut))

(95, 114, 384)
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 114
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 113
Rank of the data array before DyCA: 114
Rank of the data array be

In [26]:
# Run dyca on the first trial of each group
dyca_results = {}
for group_name, group_data in grouped_arrays.items():
    # Select the first trial (index 0)
    first_trial_data = group_data.isel(trial=0)
    
    # Convert to numpy array and ensure it's 2D (regions x time)
    data_2d = first_trial_data.values
    if data_2d.ndim == 1:
        data_2d = data_2d[np.newaxis, :]  # Add a new axis if only one region
    
    print(f"Running dyca on group: {group_name}, data shape: {data_2d.shape}")
    
    # check the rank of the data
    rank = np.linalg.matrix_rank(data_2d)
    print(f"Data rank: {rank}, shape: {data_2d.shape}")
    
    # Run dyca
    dyca_result = dyca.dyca(data_2d.T)
    dyca_results[group_name] = dyca_result
    print(f"dyca completed for group: {group_name}")

NameError: name 'grouped_arrays' is not defined